# Modernization of Locks: Pivoting on the Complexity.

In [26]:
#r "nuget:Newtonsoft.Json"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

using XPlot.Plotly;
using System.IO;
using Newtonsoft.Json;

Installed Packages Newtonsoft.Json, 13.0.3 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

In [27]:
// Root myDeserializedClass = JsonConvert.DeserializeObject<List<Root>>(myJsonResponse);
public class Datetime
{
    public string datetime { get; set; }
    public double elapsed { get; set; }
    public double delta { get; set; }
}

public class Params
{
    public double spin_count { get; set; }
}

public class Root2
{
    public double target { get; set; }
    public Params @params { get; set; }
    public Datetime datetime { get; set; }
}

## MWaitX Pivot

In [45]:
var baseline = @".\ComplexityPivot\SpinCount";
var comparand = @".\ComplexityPivot\MWaitX";

var baselineFolder = Directory.GetFiles(baseline);
var comparandFolder = Directory.GetFiles(comparand);

Dictionary<int, List<Root2>> complexityData_Baseline = new();
Dictionary<int, List<Root2>> complexityData_Comparand = new();

foreach (var f in baselineFolder)
{
    var data = File.ReadAllText(f);
    var d = JsonConvert.DeserializeObject<List<Root2>>(data);
    complexityData_Baseline[Int32.Parse(Path.GetFileNameWithoutExtension(f).Split("_")[1])] = d;
}

foreach (var f in comparandFolder)
{
    var data = File.ReadAllText(f);
    var d = JsonConvert.DeserializeObject<List<Root2>>(data);
    complexityData_Comparand[Int32.Parse(Path.GetFileNameWithoutExtension(f).Split("_")[1])] = d;
}

In [46]:
complexityData_Comparand.Keys

[ 10, 15, 20, 25, 30, 5 ]

In [47]:
void ChartComplexity(List<Root2> baseline, List<Root2> comparand, string complexity)
{
    // optima.
    var optimaY = d.Min(d => -d.target);
    var optimaX = d.FirstOrDefault(g => g.target == -optimaY).@params.spin_count;

    var layout = new Layout.Layout
    {
        xaxis = new Xaxis { title = "Default Impl vs. MWaitX" },
        yaxis = new Yaxis { title = $"Execution Time (MSec)" },
        title = $"Execution Time vs. Complexity: {complexity}", 
    };

    // Skip outlier.
    var ordered_Baseline = baseline.OrderBy(y => y.@params.spin_count).Skip(1);
    var ordered_Comparand = comparand.OrderBy(y => y.@params.spin_count).Skip(1);

    var scatter_baseline = new Scatter
    {
        x = ordered_Baseline.Select(y => (y.@params.spin_count)), 
        y = ordered_Baseline.Select(y => -y.target), 
        mode = "markers+lines",
        name = "Baseline - Explored Points"
    };

    var scatter_comparand = new Scatter
    {
        x = ordered_Comparand.Select(y => (y.@params.spin_count)), 
        y = ordered_Comparand.Select(y => -y.target), 
        mode = "markers+lines",
        name = "Comparand - Explored Points"
    };

    Chart.Plot(new [] { scatter_baseline, scatter_comparand }, layout).Display();
}

In [51]:
Console.WriteLine("Baseline Command line: --input_count 30000 --complexity {complexity} --ht 0 --thread_count 126 --join_type 1 --affi 0 --spin_count {int(spin_count)}");
Console.WriteLine("Comparand Command line: --input_count 30000 --complexity {complexity} --ht 0 --thread_count 126 --join_type 5 --affi 0 --mwaitx_cycle_count {int(spin_count)}");
foreach (var f in complexityData_Baseline.OrderBy(c => c.Key))
{
    ChartComplexity(complexityData_Baseline[f.Key], complexityData_Comparand[f.Key], f.Key.ToString());
}

Baseline Command line: --input_count 30000 --complexity {complexity} --ht 0 --thread_count 126 --join_type 1 --affi 0 --spin_count {int(spin_count)}
Comparand Command line: --input_count 30000 --complexity {complexity} --ht 0 --thread_count 126 --join_type 5 --affi 0 --mwaitx_cycle_count {int(spin_count)}


<!DOCTYPE html>

<!DOCTYPE html>

<!DOCTYPE html>

<!DOCTYPE html>

<!DOCTYPE html>

| Complexity | Minima | 
| ---------- | ------ |
| 5          |    167922    | 
| 10          |  182126      | 
| 15          |  201018      | 
| 20          | 216133       | 
| 25          |  224623      | 
| 30          |   216457     | 

In [33]:
System.Diagnostics.Process.GetCurrentProcess().Id.Display();

36904